In [1]:
# importing libraries
library(keras)
library(abind)
library(grid)

In [2]:
# loading data
mnist <- dataset_mnist()
x_train <- mnist$train$x
x_test <- mnist$test$x

In [3]:
# transforming RGB values into [0,1] range
x_train <- x_train / 255
x_test <- x_test / 255

In [4]:
cat('x_train_shape:', dim(x_train), '\n')
cat(nrow(x_train), 'train samples\n')
cat(nrow(x_test), 'test samples\n')

x_train_shape: 60000 28 28 
60000 train samples
10000 test samples


In [5]:
img_rows <- dim(x_train)[2]
img_cols <- dim(x_train)[3]

In [6]:
# redefining  dimension of train/test inputs
x_train <- array_reshape(x_train, c(nrow(x_train), 784))
x_test <- array_reshape(x_test, c(nrow(x_test), 784))


In [7]:
cat('x_train_shape:', dim(x_train), '\n')
cat(nrow(x_train), 'train samples\n')
cat('Datatype of training data :',class(x_train), '\n')

cat(nrow(x_test), 'test samples\n')
cat('Datatype of testing data :',class(x_test), '\n')


x_train_shape: 60000 784 
60000 train samples
Datatype of training data : matrix 
10000 test samples
Datatype of testing data : matrix 


In [8]:
# building an autoencoder
input_img <- layer_input(shape = c(784),name = "input")

# "encoded" is the encoded representation of the input
encoded = input_img %>%
    layer_dense(128, activation='relu',name = "encoder_1") %>%
    layer_dense(64, activation='relu',name = "encoder_2") %>%
    layer_dense(32, activation='relu',name = "encoder_3")

# "decoded" is the reconstruction of the input
decoded = encoded %>%
    layer_dense(64, activation='relu',name = "decoder_1")%>%
    layer_dense(128, activation='relu',name = "decoder_2")%>%
    layer_dense(784,activation = 'sigmoid',name = "decoder_3")

# this model maps an input to its reconstruction
autoencoder = keras_model(input_img, decoded)
summary(autoencoder)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input (InputLayer)                  (None, 784)                     0           
________________________________________________________________________________
encoder_1 (Dense)                   (None, 128)                     100480      
________________________________________________________________________________
encoder_2 (Dense)                   (None, 64)                      8256        
________________________________________________________________________________
encoder_3 (Dense)                   (None, 32)                      2080        
________________________________________________________________________________
decoder_1 (Dense)                   (None, 64)                      2112        
________________________________________________________________________________
decoder_2 (Dense)           

In [9]:
# encoder model that maps an input to its encoded representation
encoder = keras_model(input_img, encoded)
summary(encoder)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input (InputLayer)                  (None, 784)                     0           
________________________________________________________________________________
encoder_1 (Dense)                   (None, 128)                     100480      
________________________________________________________________________________
encoder_2 (Dense)                   (None, 64)                      8256        
________________________________________________________________________________
encoder_3 (Dense)                   (None, 32)                      2080        
Total params: 110,816
Trainable params: 110,816
Non-trainable params: 0
________________________________________________________________________________


In [10]:
# decoder model

# input layer for decoder
encoded_input = layer_input(shape=c(32),name = "encoded_input")

# retrieve the layer of the autoencoder model for decoder
decoder_layer1 <- get_layer(autoencoder,name= "decoder_1")
decoder_layer2 <- get_layer(autoencoder,name= "decoder_2")
decoder_layer3 <- get_layer(autoencoder,name= "decoder_3")

# create the decoder model from retreived layers
decoder =  keras_model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

In [11]:
summary(decoder)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
encoded_input (InputLayer)          (None, 32)                      0           
________________________________________________________________________________
decoder_1 (Dense)                   (None, 64)                      2112        
________________________________________________________________________________
decoder_2 (Dense)                   (None, 128)                     8320        
________________________________________________________________________________
decoder_3 (Dense)                   (None, 784)                     101136      
Total params: 111,568
Trainable params: 111,568
Non-trainable params: 0
________________________________________________________________________________


In [12]:
# compiling the model
autoencoder %>% compile(optimizer = 'adadelta',loss='binary_crossentropy')

# training the model
autoencoder %>% fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=TRUE,
                validation_data=list(x_test, x_test))

In [13]:
# encoding the test data images
encoded_imgs = encoder %>% predict(x_test)

In [14]:
# reconstructing the original input test images from the encoded representation
decoded_imgs = decoder %>% predict(encoded_imgs)

# calculating reconstruction error
reconstruction_error = metric_mean_squared_error(x_test,decoded_imgs)
paste("reconstruction error: " ,k_get_value(k_mean(reconstruction_error)))

In [ ]:
encoded_train_imgs = encoder %>% predict(x_train)

In [ ]:
# Building the model
model <- keras_model_sequential() 
model %>% 
  layer_dense(units = 256, activation = 'relu', input_shape = c(encoded_dim)) %>% 
  layer_dropout(rate = 0.4) %>% 
  layer_dense(units = 128, activation = 'relu') %>%
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 10, activation = 'softmax')

# compiling the model
model %>% compile(
  loss = 'categorical_crossentropy',
  optimizer = optimizer_rmsprop(),
  metrics = c('accuracy')
)


In [ ]:
# extracting class labels
y_train <- mnist$train$y
y_test <- mnist$test$y

# Converting class vector (integers) to binary class matrix.
y_train <- to_categorical(y_train, 10)
y_test <- to_categorical(y_test, 10)

# training the model
history <- model %>% fit(
  encoded_train_imgs, y_train, 
  epochs = 30, batch_size = 128, 
  validation_split = 0.2
)

In [ ]:
model %>% evaluate(encoded_imgs, y_test, batch_size = 128)